## 准确率、召回率、`f1`
***
***
Time: 2020-09-20
Author: dsy
***

1. 真正例(True Positive,TP):一个样本的真实类别为c并且模型正确地预测为类别c。这类样本数量记为：
$$
TP_c = \sum_{n=1}^{N} I(y^{(n)} = \hat{y}^{(n)} =c)
$$
2. 假负例（False Negative，FN）：一个样本的真实类别为c，模型错误地预测为其它类。这类样本数量记为:
$$
FN_c = \sum_{n=1}^N I( y^{(n)} =c \land \hat{y}^{(n)}  \ne c   )
$$
3. 假正例（False Positive，FP）一个样本的真实类别为其它类，模型错误地预测为类别c。这类样本数量记为:
$$
FP_c = \sum_{n=1}^N I( y^{(n)} \ne c \land \hat{y}^{(n)}  = c   )
$$
4. 真负例（True Negative，TN）：一个样本的真实类别为其它类，模型也预测为其它类。这类样本数量记为T Nc。对于类别c来说，这种情况一般不需要关注。
***
5. 查准率（Precision），也叫精确率或精度，类别c的查准率是所有预测为类别c的样本中，预测正确的比例。
$$
P_c = \frac{TP_c}{TP_c+FP_c}
$$
6. 查全率（Recall），也叫召回率，类别c的查全率是所有真实标签为类别c的样本中，预测正确的比例。
$$
R_c = \frac{TP_c}{TP_c+FN_c}
$$
7. F值（F Measure）是一个综合指标，为查准率和查全率的调和平均。
$$
F_c = \frac{(1+\beta^2) \times P_c \times R_c}{\beta^2 \times P_c + R_c}
$$
其中 β 用于平衡查全率和查准率的重要性，一般取值为 1。β = 1 时的 F 值称为F1值，是查准率和查全率的调和平均。
8. 宏平均和微平均 为了计算分类算法在所有类别上的总体查准率、查全率和 F1值，经常使用两种平均方法，分别称为宏平均（Macro Average）和微平均（Micro Average）,宏平均是每一类的性能指标的算术平均值:
$$
\begin{array}{cl}
P_{macro} & = \frac{1}{C} \sum_{c=1}^C P_c \\
R_{macro} & = \frac{1}{C} \sum_{c=1}^C R_c \\
F1_{macro} & = \frac{2 \times P_{macro} \times R_{macro}}{P_{macro} + R_{macro}}
\end{array}
$$
微平均是每一个样本的性能指标的算术平均值。对于单个样本而言，它的查
准率和查全率是相同的（要么都是1，要么都是0）。因此查准率的微平均和查全
率的微平均是相同的。同理，F1值的微平均指标是相同的。当不同类别的样本数
量不均衡时，使用宏平均会比微平均更合理些。

In [1]:
import numpy as np
from sklearn.metrics import recall_score,precision_score,f1_score

In [2]:
y_true = np.array([4, 5, 6, 4, 5, 6])
y_pred = np.array([4, 6, 5, 4, 4, 5])

### 根据公式自定义计算

In [3]:
labels = np.union1d(y_pred,y_true)
c = labels[0]
c

4

In [4]:
TP = (y_pred[y_true == c] == c).sum()
TP

2

In [5]:
FN = (y_pred[ y_true == c] != c).sum()
FN

0

In [6]:
FP = (y_pred[y_true != c] == c).sum()
FP

1

In [7]:
pmacro = 1/len(labels) * TP / (TP+FP)
pmacro

0.2222222222222222

In [8]:
rmacro = 1/len(labels) * TP / (TP+FN)
rmacro

0.3333333333333333

In [9]:
f1macro = 2 * pmacro * rmacro / (pmacro + rmacro)
f1macro

0.26666666666666666

### 使用`sklearn`进行计算`precision`,`recall`,`f1`

In [10]:
P_macro = precision_score(y_true,y_pred,average="macro")
P_macro

0.2222222222222222

In [11]:
R_macro = recall_score(y_true,y_pred,average="macro")
R_macro

0.3333333333333333

In [12]:
f1_marco = f1_score(y_true,y_pred,average="macro")
f1_marco

0.26666666666666666

In [13]:
2 * P_macro * R_macro / ( P_macro + R_macro)

0.26666666666666666